# License
    IPython notebook for simulating the heat equation with OpenCL
    Copyright (C) 2015 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Lets have opencl ipython integration enabled
%load_ext pyopencl.ipython_ext

#Import packages we need
import numpy as np
import pyopencl as cl
import os
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [2]:
#Setup easier to use compilation of OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_CTX"] = "0"
os.environ["CUDA_CACHE_DISABLE"] = "1"

In [3]:
#Create OpenCL context
cl_ctx = cl.create_some_context()

#Create an OpenCL command queue
cl_queue = cl.CommandQueue(cl_ctx)

# Heat equation in 2D
The heat equation can be written
$$
\begin{align}
\frac{\partial u}{\partial t} &= \kappa \nabla^2 u\\
&=\kappa \left[ \frac{\partial^2 u}{\partial x^2} +  \frac{\partial^2 u}{\partial y^2} \right]
\end{align}
$$
where $u$ is the temperature, and $\kappa$ is the material specific heat conduction constant. 

By approximating the temporal derivative with a backward difference, and the spatial derivative with a central difference, we get
$$
\frac{1}{\Delta t} (u_{i, j}^{n+1} - u_{i, j}^{n}) 
= \kappa \left [
\frac{1}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{1}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
\right]
$$
and gathering $u^n+1$ on the left hand side and $u^n$on the right, we write
$$
u^{n+1}_{i,j} = u_{i,j}^n 
+ \frac{\kappa\Delta t}{\Delta x^2}(u_{i-1, j}^n - 2u_{i, j}^n + u_{i+1, j}^n)
+ \frac{\kappa\Delta t}{\Delta y^2}(u_{i, j-1}^n - 2u_{i, j}^n + u_{i, j+1}^n)
$$
This discretization is unstable if the following CFL condition is not met
$$
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta x^2}, \qquad
\frac{1}{2} \gt \frac{\kappa\Delta t}{\Delta y^2}
$$
or 
$$
\Delta t \lt \text{min}\left(\frac{\Delta x^2}{2\kappa}, \frac{\Delta y^2}{2\kappa}\right)
$$

In [4]:
%%cl_kernel 
__kernel void heat_eq_2D(__global float *u1, __global const float *u0, float kappa, float dt, float dx, float dy) {
    //Get total number of cells
    int nx = get_global_size(0); 
    int ny = get_global_size(1);

    //Get position in grid
    int i = get_global_id(0); 
    int j = get_global_id(1); 
    int center = j*nx + i;

    //Internal cells
    if (i > 0 && i < nx-1 && j > 0 && j <ny-1) {
        //Calculate the four indices of our neighboring cells
        int north = (j+1)*nx + i;
        int south = (j-1)*nx + i;
        int east = j*nx + i+1;
        int west = j*nx + i-1;

        u1[center] = u0[center]
            + kappa*dt/(dx*dx) * (u0[west] - 2*u0[center] + u0[east])
            + kappa*dt/(dy*dy) * (u0[south] - 2*u0[center] + u0[north]);
    }
    //Boundary conditions (ghost cells)
    else { 
        u1[center] = u0[center];
    }
}

In [5]:
"""
Class that holds data for the heat equation in OpenCL
"""
class HeatDataCL:
    """
    Uploads initial data to the CL device
    """
    def __init__(self, u0):
        #Make sure that the data is single precision floating point
        assert(np.issubdtype(u0.dtype, np.float32))
        assert(np.isfortran(u0))
        
        #Find number of cells
        self.nx = u0.shape[0]
        self.ny = u0.shape[1]
        
        mf = cl.mem_flags 
        
        #Upload data to the device
        self.u0 = cl.Buffer(cl_ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=u0)
        
        #Allocate output buffers
        self.u1 = cl.Buffer(cl_ctx, mf.READ_WRITE, u0.nbytes)
        
    """
    Enables downloading data from CL device to Python
    """
    def download(self):
        #Allocate data on the host for result
        u0 = np.empty((self.nx, self.ny), dtype=np.float32, order='F')
        
        #Copy data from device to host
        cl.enqueue_copy(cl_queue, u0, self.u0) 
        
        #Return
        return u0;

In [6]:
"""
Computes the heat equation using an explicit finite difference scheme with OpenCL
"""
def opencl_heat_eq(cl_data, kappa, dx, dy, nt):
    #Calculate dt from the CFL condition
    dt = 0.4 * min(dx*dx / (2.0*kappa), dy*dy / (2.0*kappa))

    #Loop through all the timesteps
    for i in range(0, nt):
        #Execute program on device
        heat_eq_2D(cl_queue, (cl_data.nx, cl_data.ny), None, \
                   cl_data.u1, cl_data.u0, \
                   np.float32(kappa), np.float32(dt), np.float32(dx), np.float32(dy))
        
        #Swap variables
        cl_data.u0, cl_data.u1 = cl_data.u1, cl_data.u0

In [12]:
#Create test input data
u0 = np.random.rand(100, 50);
u0 = np.asfortranarray(u0, dtype=np.float32)
cl_data = HeatDataCL(u0)
kappa = 1.0
dx = 1.0
dy = 2.0

#Plot initial conditions
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.suptitle("Heat equation 2D", fontsize=18)

max_x = cl_data.nx*dx
max_y = cl_data.ny*dy

x, y = np.mgrid[0:max_x:dx, 0:max_y:dy]

surf = ax.plot_surface(x, y, u0, cmap=cm.coolwarm, shade=True, vmin=0.0, vmax=1.0, cstride=1, rstride=1)
ax.set_zlim(0.0, 1.0)

#for i in range(1, 9):
#    timesteps_per_plot=5
#    #Simulate 10 timesteps
#    opencl_heat_eq(cl_data, kappa, dx, dy, timesteps_per_plot)

    #Download data
#    u1 = cl_data.download()

    #Plot
#    ax_2d[i].imshow(easy_surf(u1, dx, dy))
#    ax_2d[i].axis('off')


def animate(i):
    timesteps_per_plot=1
    
    if (i>0):
        opencl_heat_eq(cl_data, kappa, dx, dy, timesteps_per_plot)
        
    u1 = cl_data.download()
    ax.clear()
    ax.plot_surface(x, y, u1, cmap=cm.coolwarm, shade=True, vmin=0.0, vmax=1.0, cstride=1, rstride=1)
            
anim = animation.FuncAnimation(fig, animate, range(50), interval=100)
plt.close(anim._fig)
anim